In [3]:
#Wai Chin Kang
#SW01082417

In [1]:
import pandas as pd

# Load the uploaded CSV file
file_path = "news_dataset.csv"
df = pd.read_csv(file_path)

# Display the first few rows and the column names to verify the structure
df.head(), df.columns

(   Unnamed: 0                                               text  target  \
 0           0  I was wondering if anyone out there could enli...       7   
 1          17  I recently posted an article asking what kind ...       7   
 2          29  \nIt depends on your priorities.  A lot of peo...       7   
 3          56  an excellent automatic can be found in the sub...       7   
 4          64  : Ford and his automobile.  I need information...       7   
 
        title                        date  
 0  rec.autos  2022-08-02 13:48:37.251043  
 1  rec.autos  2022-08-02 13:48:37.251043  
 2  rec.autos  2022-08-02 13:48:37.251043  
 3  rec.autos  2022-08-02 13:48:37.251043  
 4  rec.autos  2022-08-02 13:48:37.251043  ,
 Index(['Unnamed: 0', 'text', 'target', 'title', 'date'], dtype='object'))

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from gensim.utils import simple_preprocess
import re

# Manually append path
nltk.data.path.append(r'C:\Users\User\AppData\Roaming\nltk_data')

# Download (if not yet done)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Init
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Clean text
def preprocess(text):
    tokens = simple_preprocess(str(text), deacc=True)  # deacc=True removes punctuations
    tokens = [lemmatizer.lemmatize(stemmer.stem(word)) for word in tokens if word not in stop_words and len(word) > 2]
    return tokens

# Apply
df = df[['text']].dropna()
df['text'] = df['text'].astype(str)
df['tokens'] = df['text'].apply(preprocess)
df['tokens'].head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    [wonder, anyon, could, enlighten, car, saw, da...
1    [recent, post, articl, ask, kind, rate, singl,...
2    [depend, prioriti, lot, peopl, put, higher, pr...
3    [excel, automat, found, subaru, legaci, switch...
4    [ford, automobil, need, inform, whether, ford,...
Name: tokens, dtype: object

In [18]:
from gensim.corpora import Dictionary
from gensim import models
from gensim.models import CoherenceModel

df_small = df.head(1000)

# Create Dictionary and Corpus from your preprocessed data
id2word_small = Dictionary(df_small['tokens'])
corpus_small = [id2word_small.doc2bow(text) for text in df_small['tokens']]

# Train LDA model
lda_model_small = models.LdaModel(corpus=corpus_small,
                                  id2word=id2word_small,
                                  num_topics=4,
                                  random_state=42,
                                  update_every=1,
                                  chunksize=100,
                                  passes=10,
                                  alpha='auto',
                                  per_word_topics=True)
# Display the topics
lda_topics = lda_model.print_topics(num_words=10)
lda_topics

coherence_model_lda = CoherenceModel(model=lda_model_small,
                                     texts=df_small['tokens'],
                                     dictionary=id2word_small,
                                     coherence='c_v')

coherence_score = coherence_model_lda.get_coherence()
print(f"Coherence Score: {coherence_score}")

Coherence Score: 0.4510210271635


In [20]:
#Interpretation: The Coherence Score reflects how semantically interpretable the topics are. 
#Scores closer to 1.0 suggest clear and meaningful topic separation. 
#For this model, a coherence score above 0.4 is typically acceptable, while 0.5–0.6 indicates good semantic consistency. 
#This helps determine if the 4 topics discovered are reasonably coherent based on the keywords.